In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

from pyquil.api import get_qc

ideal_qc = get_qc('4q-qvm', noisy=False)
noisy_qc = get_qc("4q-noisy-qvm", noisy=True)

qubits = ideal_qc.qubits()
print(qubits)
graph = ideal_qc.qubit_topology()
nx.draw_networkx(graph, with_labels=True)

## Import and setup logging for info on progress.

In [ ]:
from forest_benchmarking.quantum_volume import measure_quantum_volume

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
show_progress_bar = True

## Replicate FIG2. of [QVol]  (See forest_qcvv.quantum_volume for ref)

This is SLOW--it takes about 4 minutes.

In [ ]:
ideal_outcomes = measure_quantum_volume(ideal_qc, num_circuits=200, show_progress_bar=show_progress_bar)

## Now with noise

This is SLOW--it takes about 5 minutes, even with half the number of shots from above.

In [ ]:
noisy_outcomes = measure_quantum_volume(noisy_qc, num_circuits=200, num_shots=500, show_progress_bar=show_progress_bar)

In [ ]:
depths = np.arange(2, 5)
ideal_probs = [val[1] for val in ideal_outcomes]
noisy_probs = [val[1] for val in noisy_outcomes]

plt.axhline(.5 + np.log(2)/2, color='b', ls='--', label='ideal asymptote')
plt.axhline(2/3, color='black', ls=':', label='achievable threshold')
plt.scatter(np.array(depths) - .1, ideal_probs, color='b', label='ideal simulation')
plt.scatter(depths, noisy_probs, color='r', label='noisy simulation')
plt.ylabel("est. heavy output probability h_d")
plt.xlabel("width/depth of model circuit m=d")
plt.ylim(.4,1.0)
plt.xlim(1.8, 4.2)
plt.xticks(depths)
plt.legend(loc='lower left')
plt.show()

## Try a different topology

In [ ]:
n = 4
path_graph = nx.path_graph(n)
loop_graph = nx.cycle_graph(n)
four_pointed_star = nx.star_graph(n)

In [ ]:
nx.draw_networkx(path_graph, with_labels=True)

In [ ]:
nx.draw_networkx(loop_graph, with_labels=True)

In [ ]:
nx.draw_networkx(four_pointed_star, with_labels=True)

In [ ]:
from pyquil.api._quantum_computer import _get_qvm_with_topology
path_qc = _get_qvm_with_topology(name='path', topology=path_graph, noisy=True)

In [ ]:
path_outcomes = measure_quantum_volume(path_qc, num_circuits=200, num_shots=500, show_progress_bar=show_progress_bar)

## Compare to noisy complete topology

In [ ]:
depths = np.arange(2, 5)
ideal_probs = [val[1] for val in ideal_outcomes]
noisy_probs = [val[1] for val in noisy_outcomes]
path_probs = [val[1] for val in path_outcomes]

plt.axhline(.5 + np.log(2)/2, color='b', ls='--', label='ideal asymptote')
plt.axhline(2/3, color='grey', ls=':', label='achievable threshold')
plt.scatter(np.array(depths) - .1, ideal_probs, color='b', label='ideal simulation')
plt.scatter(depths, noisy_probs, color='r', label='noisy complete simulation')
plt.scatter(depths, path_probs, color='black', label='noisy path simulation')
plt.ylabel("est. heavy output probability h_d")
plt.xlabel("width/depth of model circuit m=d")
plt.ylim(.4,1.0)
plt.xlim(1.8, 4.2)
plt.xticks(depths)
plt.legend(loc='lower left')
plt.show()

## For more fine-grained control, create and maintain a dataframe

In [ ]:
from forest_benchmarking.quantum_volume import (generate_quantum_volume_experiments,
                                                add_programs_to_dataframe,
                                                acquire_quantum_volume_data,
                                                acquire_heavy_hitters,
                                                get_results_by_depth,
                                                extract_quantum_volume_from_results)

Get a dataframe with (depth x n_circuits) many "Abstract Ckt"s that describe each model circuit for each depth.

In [ ]:
n_circuits = 100
depths = [2,3]
df = generate_quantum_volume_experiments(depths, n_circuits)
df

Use the default program_generator to synthesize native pyquil programs that implement each ckt natively on the qc.

In [ ]:
df = add_programs_to_dataframe(df, noisy_qc)
print(df["Program"].values[0])

Run the programs. This can be slow.

In [ ]:
df = acquire_quantum_volume_data(df, noisy_qc, num_shots=10)

Classically simulate the circuits to get heavy hitters, and record how many hh were sampled for each program run.

In [ ]:
df = acquire_heavy_hitters(df)
print(df["Num HH Sampled"].values[0:10])

Get estimates of the probability of sampling hh at each depth, and the lowerbound on that estimate

In [ ]:
results = get_results_by_depth(df)
results

Use the results to get a lower bound on the quantum volume

In [ ]:
qv = extract_quantum_volume_from_results(results)
qv